In [ ]:
# default_exp model.diagnosis

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from fastcore.test import *

# Diagnosis

> Structure for performing Diagnosis for 3D Datasets

<img src="../img/diagnosis.png" alt="Drawing" style="width: 600px;"/>
     

     
https://stanfordmlgroup.github.io/projects/mrnet/

In [ ]:
# export

import pdb
import torch
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F


from torchvision import models
from deeptool.utils import Tracker

## 1. The Final Classification Networks

In [ ]:
# export

class Classify(nn.Module):
    """
    The Classifier on top of the triplenet network
    """

    def __init__(self, in_dim, mid_dim, out_dim, p_drop=0.5):
        """init the classifier"""
        super(Classify, self).__init__()
        # reduction block
        self.reduce = nn.Sequential(
            nn.Linear(in_dim, mid_dim),
            nn.Dropout(p=p_drop),
            nn.ReLU(inplace=True),
        )
        # final block
        self.fin_block = nn.Sequential(
            nn.Linear(mid_dim, out_dim)
        )

    def forward(self, x):
        """perform forward calculation"""
        # reduce
        x = self.reduce(x)
        x = self.fin_block(x)
        return x

## Proposed extension: Maintaining the spatial Relation using Recurrence

<img src="../img/diagnosis_rnn.png" alt="Drawing" style="width: 800px;"/>

In [ ]:
# export

class Classify_RNN(nn.Module):
    """
    The Classifier on top of the triplenet network
    """

    def __init__(self, device, input_size, hidden_size, n_layers=1):
        """init the classifier"""
        super(Classify_RNN, self).__init__()
        self.device = device
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_size, n_layers).to(self.device)

    def init_hidden(self):
        """create zeros for hidden layer"""
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

    def forward(self, x):
        """perform forward calculation"""
        x = x.view(-1, 1, self.input_size)
        # create hidden dimension
        hidden = self.init_hidden()
        # apply GRU
        _, hidden = self.gru(x, hidden)
        # use hidden for final linear layer (equal to last output)
        return hidden.view(1, -1)

## 2. The Complete Network with Backbone and Classifier

In [ ]:
# export


class TripleMRNet(nn.Module):
    """
    adapted from https://github.com/yashbhalgat/MRNet-Competition
    with the knowledge of: https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1002699
    """

    def __init__(self, device, args, train_data=None, backbone="resnet18", training=True, document=True):
        super(TripleMRNet, self).__init__()

        # general defines
        self.device = device
        self.backbone = args.mrnet_backbone
        self.y_labels = args.classes
        self.y_len = len(self.y_labels)
        self.naming = args.perspectives

        # depending on whether train_data is specified
        if train_data == None:
            self.weights = {}
            self.weights['abn'] = [0.81, 0.19]
            self.weights['acl'] = [0.23, 0.77]
            self.weights['men'] = [0.43, 0.57]
        else:
            self.weights = train_data.weights

        # picture center cropping to 224 resolution
        self.pic_size = 224
        self.pad = int((args.pic_size - self.pic_size) / 2)
        self.factor = 1  # 1130 / 208  # inverse factor

        # internal count for updating
        self.int_count = 0
        self.batch_update = args.mrnet_batch_update
        self.label_smoothing = args.mrnet_label_smoothing

        # Setup the tracker to visualize the progress
        if args.track:
            self.tracker = Tracker(args)

        # average together
        self.rnn_gap = args.mrnet_rnn_gap
        self.hidden_dim = args.mrnet_hidden_dim

        # define whether to use one network or multiple ones
        self.single_stream = args.mrnet_singlestream

        # build backbone networks
        self.axial_net = self.build_backbone(training)
        self.sagit_net = self.build_backbone(training)
        self.coron_net = self.build_backbone(training)

        # build gap and classifier
        # apply average pooling
        self.gap_axial = nn.AdaptiveAvgPool2d(1).to(self.device)
        self.gap_sagit = nn.AdaptiveAvgPool2d(1).to(self.device)
        self.gap_coron = nn.AdaptiveAvgPool2d(1).to(self.device)

        if self.rnn_gap:
            # only for the avergae case
            if self.backbone in ("resnet18", "vgg", "squeeze"):
                self.input_rnn = 512
            else:
                self.input_rnn = 256

            # the RNN gapping
            self.max_axial = Classify_RNN(
                self.device, self.input_rnn, self.hidden_dim)
            self.max_sagit = Classify_RNN(
                self.device, self.input_rnn, self.hidden_dim)
            self.max_coron = Classify_RNN(
                self.device, self.input_rnn, self.hidden_dim)

            # add a classifier at the end
            #self.classifier = nn.Sequential(nn.Linear(3 * self.hidden_dim, self.y_len))
            self.classifier = Classify(
                3*self.hidden_dim, self.hidden_dim, self.y_len)

        else:
            # only for the avergae case
            if self.backbone in ("resnet18", "vgg", "squeeze"):
                self.classifier = Classify(
                    3*512, self.hidden_dim, self.y_len).to(self.device)
            elif self.backbone == "alexnet":
                self.classifier = Classify(
                    3*256, self.hidden_dim, self.y_len).to(self.device)

        # fineal sigmoid layer
        self.sigmoid = nn.Sigmoid().to(self.device)

        # redefine if single stream
        if self.single_stream:
            # make them all reference the same network
            self.sagit_net = self.axial_net
            self.coron_net = self.axial_net

            if self.rnn_gap:
                self.max_sagit = self.max_axial
                self.max_coron = self.max_axial

        self.classifier.to(self.device)
        self.axial_net.to(self.device)
        self.coron_net.to(self.device)
        self.sagit_net.to(self.device)

        # define optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=args.lr)

    def build_backbone(self, training):
        """
        Builds the desired backbone
        """
        if self.backbone == "resnet18":
            resnet = models.resnet18(pretrained=training)
            modules = list(resnet.children())[:-1]
            local_net = nn.Sequential(*modules)
            for param in local_net.parameters():
                param.requires_grad = False

        elif self.backbone == "alexnet":
            local_net = models.alexnet(pretrained=training)
            local_net = local_net.features

        elif self.backbone == "vgg":
            local_net = models.vgg11(pretrained=training)
            local_net = local_net.features

        elif self.backbone == "squeeze":
            local_net = models.squeezenet1_1(pretrained=training)
            local_net = local_net.features

        return local_net

    def weighted_loss(self, prediction, target, cl):
        """
        Calculate the weighted loss with label smoothing 
        """
        # determin the weights
        weights_npy = np.array(
            [self.weights[cl][int(target)]])
        weights_tensor = torch.FloatTensor(weights_npy)
        weights_tensor = weights_tensor.to(self.device)[0]

        # smooth the labels
        if self.label_smoothing > 0:
            target = target.add(self.label_smoothing).div(2)

        # calculate binary cross entropy
        loss = F.binary_cross_entropy_with_logits(
            prediction, target, weight=Variable(weights_tensor))
        return loss

    def watch_progress(self, valid_loader, iteration):
        """Outsourced to Tracker"""
        self.tracker.get_accuracy(self, valid_loader, iteration)

    def get_input_image(self, data):
        """
        take the input from the stack and give the single volumes
        """
        # get the three volumes from the dictionary
        # data["img"]["axial"] -> shape = batch x depth x pic x pic
        vol_axial = data["img"][self.naming[0]][0, :, :, :].to(self.device)
        vol_axial = torch.stack((vol_axial,)*3, axis=1)

        vol_sagit = data["img"][self.naming[1]][0, :, :, :].to(self.device)
        vol_sagit = torch.stack((vol_sagit,)*3, axis=1)

        vol_coron = data["img"][self.naming[2]][0, :, :, :].to(self.device)
        vol_coron = torch.stack((vol_coron,)*3, axis=1)

        label = torch.zeros(vol_axial.shape[0], self.y_len)  # init
        for i, cl in enumerate(self.y_labels):
            label[:, i] = data[cl]
        label = label[0, :].to(self.device)

        return vol_axial, vol_sagit, vol_coron, label

    def apply_gap(self, vol_axial, vol_sagit, vol_coron):
        """
        applies the average / rnn gap
        """
        vol_axial = self.gap_axial(vol_axial).view(vol_axial.size(0), -1)
        vol_sagit = self.gap_sagit(vol_sagit).view(vol_sagit.size(0), -1)
        vol_coron = self.gap_coron(vol_coron).view(vol_coron.size(0), -1)

        if self.rnn_gap:
            # idea add spatial relation here
            x = self.max_axial(vol_axial)[0]
            y = self.max_sagit(vol_sagit)[0]
            z = self.max_coron(vol_coron)[0]

            w = torch.cat((x, y, z), 0)

        else:
            x = torch.max(vol_axial, 0, keepdim=True)[0]
            y = torch.max(vol_sagit, 0, keepdim=True)[0]
            z = torch.max(vol_coron, 0, keepdim=True)[0]

            w = torch.cat((x, y, z), 1)[0]

        return w

    def forward(self, data, update=True):
        """
        perform the forward pass and update
        """
        # get the required input
        vol_axial, vol_sagit, vol_coron, label = self.get_input_image(data)

        # apply the main networks
        vol_axial = self.axial_net(vol_axial)
        vol_sagit = self.sagit_net(vol_sagit)
        vol_coron = self.coron_net(vol_coron)

        # apply the gap
        w = self.apply_gap(vol_axial, vol_sagit, vol_coron)

        logit = self.classifier(w)
        # accumulate losses
        loss = 0
        for i, cl in enumerate(self.y_labels):
            loss += self.weighted_loss(logit[i], label[i], cl)
        loss /= self.batch_update

        out = self.sigmoid(logit)

        if update:
            loss.backward()
            self.int_count += 1

            if self.int_count > self.batch_update:
                # finally take the update step
                self.int_count = 0
                self.optimizer.step()
                self.zero_grad()

            return out
        else:
            tr_data = {}
            tr_data["loss"] = loss.item()
            return out, label, tr_data

In [ ]:
# hide
from deeptool.train_loop import test_one_batch
from deeptool.parameters import get_all_args, compat_args
args = get_all_args()

In [ ]:
# 3 dim test
args.model_type = "diagnosis"
args.dim = 3
args = compat_args(args)
test_one_batch(args)

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_rnn_vae.ipynb.
Converted 99_index.ipynb.
